In [1]:
class SpecialIterator:
    def __init__(self, limit):
        self.limit = limit
        self.counter = 0

    def __iter__(self):
        print("iter")
        return self
        
    def __next__(self):
        print("next")
        if self.counter < self.limit:
            self.counter += 1
            return self.counter
        else:
            raise StopIteration

In [2]:
spec = SpecialIterator(5)

In [3]:
next(spec)

next


1

In [4]:
iter(spec) is spec

iter


True

In [5]:
for i in spec:
    print(f"{i=}")

iter
next
i=2
next
i=3
next
i=4
next
i=5
next


In [6]:
for i in [1, 2, 3]:
    print(i)

1
2
3


In [8]:
def empty_gen():
    print("run empty_gen")
    if False:
        yield
    return 99

In [9]:
empty = empty_gen()

In [11]:
next(empty)

run empty_gen


StopIteration: 99

In [12]:
def grep_gen(pattern):
    print("start grep for", pattern)
    i = 0
    while True:
        s = yield i
        i += 1
        if pattern in s:
            print("found!", s)
        else:
            print("no %s in %s" % (pattern, s))

In [22]:
grep = grep_gen("python")

In [23]:
grep.send(None)

start grep for python


0

In [24]:
grep.send("qwerty")

no python in qwerty


1

In [25]:
grep.send("qwerty")

no python in qwerty


2

In [26]:
grep.send("pyt")

no python in pyt


3

In [27]:
grep.send("python")

found! python


4

In [28]:
grep.send("python")

found! python


5

In [29]:
grep.throw(ValueError("wrong"))

ValueError: wrong

In [34]:
def step_counter(start, stop, step):
    print("start step_counter")

    while start < stop:
        step_cand = yield start
        if step_cand is not None:
            step = step_cand
        start += step

    print("finish  step_counter")

In [36]:
step = step_counter(0, 20, 3)

print(111, next(step))

print(222, step.send(1))
print(333, next(step))
print(444, next(step))

print(555, step.send(6))
print(666, next(step))

start step_counter
111 0
222 1
333 2
444 3
555 9
666 15


In [37]:
next(step)

finish  step_counter


StopIteration: 

In [38]:
it = iter(range(10))

result = list(
    zip(
        it,
        map(lambda x: x * x, it)
    )
)
print(result)

[(0, 1), (2, 9), (4, 25), (6, 49), (8, 81)]


In [39]:
lst = [1, 2, 3, 4, 5, 6]

In [40]:
it1 = iter(lst)

In [41]:
next(it1)

1

In [42]:
it2 = iter(lst)

In [43]:
next(it2)

1

In [44]:
next(it2)

2

In [46]:
it3 = iter(it1)

In [47]:
next(it3)

2

In [48]:
it3 is it1

True

In [49]:
import asyncio
import time
import aiohttp

In [51]:
def async_timeit(fn):
    async def inner(*args, **kwargs):
        t1 = time.time()
        try:
            return await fn(*args, **kwargs)
        finally:
            t2 = time.time()
            print(f"Time for {fn.__name__}: {t2 - t1}")
    return inner

In [56]:
URL = "https://docs.python.org/3/whatsnew/3.14.html"
URLS = [URL] * 20
WORKERS = 10


async def fetch_url(url, session):
    await asyncio.sleep(1)
    #async with session.get(url) as resp:
    #    return await resp.text()


async def fetch_worker(que, session, worker):
    print(f"start {worker=}")
    while True:
        url = await que.get()

        try:
            await fetch_url(url, session)
        except Exception:
            pass
        finally:
            que.task_done()

    print(f"finish {worker=}")


@async_timeit
async def run():
    que = asyncio.Queue()
    for url in URLS:
        await que.put(url)

    async with aiohttp.ClientSession() as session:
        workers = [
            asyncio.create_task(
                fetch_worker(que, session, f"worker_{i}")
            )
            for i in range(WORKERS)
        ]
        await que.join()

        for worker in workers:
            worker.cancel()


await run()

start worker='worker_0'
start worker='worker_1'
start worker='worker_2'
start worker='worker_3'
start worker='worker_4'
start worker='worker_5'
start worker='worker_6'
start worker='worker_7'
start worker='worker_8'
start worker='worker_9'
Time for run: 2.0021960735321045


In [57]:
URL = "https://docs.python.org/3/whatsnew/3.14.html"
URLS = [URL] * 20
WORKERS = 10


async def fetch_url(url, session, sem):
    async with sem:
        await asyncio.sleep(1)
        #async with session.get(url) as resp:
        #    return await resp.text()


async def fetch_worker(que, session, sem, worker):
    print(f"start {worker=}")
    while True:
        url = await que.get()

        try:
            await fetch_url(url, session, sem)
        except Exception:
            pass
        finally:
            que.task_done()

    print(f"finish {worker=}")


@async_timeit
async def run():
    que = asyncio.Queue()
    for url in URLS:
        await que.put(url)

    sem = asyncio.Semaphore(5)

    async with aiohttp.ClientSession() as session:
        workers = [
            asyncio.create_task(
                fetch_worker(que, session, sem, f"worker_{i}")
            )
            for i in range(WORKERS)
        ]
        await que.join()

        for worker in workers:
            worker.cancel()


await run()

start worker='worker_0'
start worker='worker_1'
start worker='worker_2'
start worker='worker_3'
start worker='worker_4'
start worker='worker_5'
start worker='worker_6'
start worker='worker_7'
start worker='worker_8'
start worker='worker_9'
Time for run: 4.005996942520142


In [62]:
URL = "https://docs.python.org/3/whatsnew/3.14.html"
URLS = [URL] * 10


async def fetch_url(url, session, name):
    await asyncio.sleep(1)
    # async with session.get(url) as resp:
    #     return await resp.text()
    return (name, 200)


async def gen_fetch_batch_urls(urls):
    async with aiohttp.ClientSession() as session:
        tasks = [
            fetch_url(url, session, f"fetch_{i}")
            for i, url in enumerate(urls)
        ]

        async for task in asyncio.as_completed(tasks):
            res = await task
            #print(f"get got {res=}")
            if res[0][-1] in "02468":
                yield res


@async_timeit
async def run():
    async for item in gen_fetch_batch_urls(URLS):
        print(f"total {item=}")


await run()

total item=('fetch_4', 200)
total item=('fetch_2', 200)
total item=('fetch_8', 200)
total item=('fetch_6', 200)
total item=('fetch_0', 200)
Time for run: 1.0019540786743164
